In [312]:
import datetime
import locale
import os
import docx
from docx import Document
from docxcompose.composer import Composer
import shutil

# Mengatur locale ke bahasa Indonesia
locale.setlocale(locale.LC_TIME, 'id_ID.utf8')

def clear_directory(directory_path):
    # Cek apakah direktori ada
    if not os.path.exists(directory_path):
        print(f"Direktori {directory_path} tidak ada.")
        return
    
    # Loop melalui semua file dan folder dalam direktori
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        
        try:
            # Jika file_path adalah sebuah file, hapus file tersebut
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            # Jika file_path adalah sebuah folder, hapus folder beserta isinya
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Gagal menghapus {file_path}. Sebab: {e}")
    
    print(f"Semua file dan folder dalam direktori {directory_path} telah dihapus.")

def string_to_date(date_str):
    """
    Konversi string tanggal berbahasa Indonesia ke objek datetime.
    """
    return datetime.datetime.strptime(date_str, "%d %B %Y")

def date_to_string(date_obj):
    """
    Konversi objek datetime ke string tanggal berbahasa Indonesia.
    """
    return date_obj.strftime("%d %B %Y")

def generate_weekly_dates(start_date_str):
    """
    Menghasilkan daftar tanggal mulai dari satu minggu ke depan hingga 14 minggu ke depan dari tanggal input.
    """
    # Parsing input string menjadi objek datetime
    start_date = string_to_date(start_date_str)
    
    # Inisialisasi daftar tanggal
    weekly_dates = []
    
    # Menambahkan tanggal-tanggal ke dalam daftar
    for i in range(15):
        current_date = start_date + datetime.timedelta(weeks=i)
        weekly_dates.append(date_to_string(current_date))
    
    return weekly_dates

def n_generate_weekly_dates(start_date_str, weeks_n):
    """
    Menghasilkan daftar tanggal mulai dari satu minggu ke depan hingga 14 minggu ke depan dari tanggal input.
    """
    # Parsing input string menjadi objek datetime
    start_date = string_to_date(start_date_str)
    
    # Inisialisasi daftar tanggal
    weekly_dates = []
    
    # Menambahkan tanggal-tanggal ke dalam daftar
    for i in range(weeks_n):
        current_date = start_date + datetime.timedelta(weeks=i)
        weekly_dates.append(date_to_string(current_date))
    
    return weekly_dates

def format_replacement(nama_dosen, jabatan_akademik, matkul, pertemuan, bulan_pertemuan, tanggal_senin, replacements_data_kelas):
    replacements = {
        "NAMA_DOSEN": nama_dosen,
        "JABATAN_AKADEMIK": jabatan_akademik,
        "MATA_KULIAH": matkul,
        "PERTEMUAN_": str(pertemuan),
        "BULAN_PERTEMUAN": bulan_pertemuan,
        "TANGGAL_SENINNYA": tanggal_senin,
    }

    for index_, value_ in replacements_data_kelas.items():
        replacements[index_] = value_
    
    return replacements



def replace_text_in_paragraph(paragraph, replacements):
    for placeholder, replacement in replacements.items():
        if placeholder in paragraph.text:
            for run in paragraph.runs:
                if placeholder in run.text:
                    run.text = run.text.replace(placeholder, replacement)

def replace_text_in_table(table, replacements):
    for row in table.rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                replace_text_in_paragraph(paragraph, replacements)

def replace_placeholder(docx_path, output_path, replacements):
    # Buka dokumen
    doc = docx.Document(docx_path)
    
    # Loop melalui semua paragraf dalam dokumen
    for paragraph in doc.paragraphs:
        replace_text_in_paragraph(paragraph, replacements)

    # Loop melalui semua tabel dalam dokumen
    for table in doc.tables:
        replace_text_in_table(table, replacements)
    
    # Simpan dokumen yang telah dimodifikasi ke output path
    doc.save(output_path)


# # tanpa page break
# def combine_docx(file_list, output_path):
#     # Fungsi untuk menggabungkan dua dokumen
#     def append_docx(master_doc, append_doc):
#         for element in append_doc.element.body:
#             master_doc.element.body.append(element)

#     # Buka dokumen pertama dan gunakan sebagai dokumen master
#     master_doc = Document(file_list[0])

#     # Loop melalui file-file lain dan gabungkan ke master_doc
#     for file_path in file_list[1:]:
#         append_doc = Document(file_path)
#         append_docx(master_doc, append_doc)
    
#     # Simpan dokumen yang telah digabungkan ke output path
#     master_doc.save(output_path)

# Dengan Page Break
def combine_docx(file_list, output_path):
    def append_docx(master_doc, append_doc):
        for element in append_doc.element.body:
            master_doc.element.body.append(element)

    # Buka dokumen pertama dan gunakan sebagai dokumen master
    master_doc = Document(file_list[0])
    master_doc.add_page_break()

    # Loop melalui file-file lain dan gabungkan ke master_doc
    for file_path in file_list[1:]:
        append_doc = Document(file_path)
        append_doc.add_page_break()
        append_docx(master_doc, append_doc)
    
    # Simpan dokumen yang telah digabungkan ke output path
    master_doc.save(output_path)

def sort_dict_by_key(input_dict):
    """
    Mengurutkan dictionary berdasarkan kunci integer.

    Args:
    input_dict (dict): Dictionary dengan kunci bertipe integer.

    Returns:
    dict: Dictionary yang telah diurutkan berdasarkan kunci.
    """
    # Menggunakan sorted() untuk mengurutkan dictionary berdasarkan kunci
    sorted_dict = dict(sorted(input_dict.items(), key=lambda item: item[0]))
    return sorted_dict


# Data penggantian
replacements = {
    "NAMA_DOSEN": "Dr. John Doe",
    "JABATAN_AKADEMIK": "Profesor",
    "MATA_KULIAH": "Pemrograman Lanjut",
    "PERTEMUAN_": "Pertemuan 3",
    "BULAN_PERTEMUAN": "Maret",
    "TANGGAL_SENINNYA": "27 Maret 2023"
}

# # Path ke file DOCX asli dan output file DOCX yang baru
# docx_path = 'DHD Praktikum Tabel 1 baris.docx'
# output_path = 'TEMP/DHD Praktikum Tabel 1 baris.docx'

# # Melakukan penggantian dan menyimpan dokumen baru
# replace_placeholder(docx_path, output_path, replacements)


In [313]:
# Input tanggal
input_date = "26 Februari 2024"

# Menghasilkan daftar tanggal mingguan
output_dates = generate_weekly_dates(input_date)

# Menampilkan hasil
print(output_dates)

['26 Februari 2024', '04 Maret 2024', '11 Maret 2024', '18 Maret 2024', '25 Maret 2024', '01 April 2024', '08 April 2024', '15 April 2024', '22 April 2024', '29 April 2024', '06 Mei 2024', '13 Mei 2024', '20 Mei 2024', '27 Mei 2024', '03 Juni 2024']


In [314]:
# Ekstrak Materi "KECERDASAN BUATAN" dari file PDF
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        num_pages = len(reader.pages)
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text += page.extract_text()
    return text

# Contoh penggunaan
pdf_path = "KECERDASAN BUATAN/Bab 2 - MODUL PRAKTIKUM.pdf"
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text.split('\n')[2])

PERTEMUAN  2 – METODE PENCARIAN  


In [315]:
materi_kecerdasan_buatan = []
for i in range(1, 11):
    materi = extract_text_from_pdf(f"KECERDASAN BUATAN/Bab {i} - MODUL PRAKTIKUM.pdf").split("\n")
    print(materi)

    if i <= 2:
        materi_kecerdasan_buatan.append(materi[2])
    if i == 3:
        materi_kecerdasan_buatan.append(materi[7:8])
    if i == 4:
        materi_kecerdasan_buatan.append(materi[7:9])
    if i == 5:
        materi_kecerdasan_buatan.append("Latihan Studi Kasus".upper())
    if i >= 6 and i <= 9:
        materi_kecerdasan_buatan.append(materi[7:10])
    if i == 10:
        materi_kecerdasan_buatan.append(materi[7:9])

[' ', 'MODUL PRAKTIKUM KECERDASAN BUATAN  ', 'PERTEMUAN  1 – REPRESENTASI PENGETAHUAN  ', ' ', 'Tools  : Jupyter notebook  ', 'Bahasa  Pemrograman  : Python  ', ' ', ' ', '1. Pengenalan  Python  dan Google  Colab  ', ' Python1 didesain oleh Guido van Rossum2 pada 1991 sebagai general -purpose language ', 'dan bersifat open -source. Python merupakan bahasa pemrograman populer tetapi juga memiliki ', 'properti yang sangat baik untuk programmer pemula, sehingga ideal untuk orang yang belum ', 'pernah mem program sebelumnya. Python juga bersifat cross -platform, artinya bisa digunakan ', 'pada sistem operasi Windows, Mac OS, dan Linux.  ', ' Beberapa kelebihan dari properti tersebut adalah kode yang mudah dibaca, pengetikan dan ', 'penggunaan memori yang dinamis. Python adalah bahasa interpreter, sehingga kode dieksekusi ', 'segera di konsol Python tanpa memerlukan langkah kompilasi ke bahasa mesin. Selain konsol ', 'Python (yang disertakan dengan instalasi Python apa pun), Anda dapat mene

In [316]:
materi_kecerdasan_buatan

['PERTEMUAN  1 – REPRESENTASI PENGETAHUAN  ',
 'PERTEMUAN  2 – METODE PENCARIAN  ',
 ['KETIDAKP ASTIAN'],
 ['LOGIKA', 'FUZZY'],
 'LATIHAN STUDI KASUS',
 ['JARINGAN', 'SY ARAF', 'TIRUAN'],
 ['ALGORITMA', 'GENETIKA', 'Tools'],
 ['SISTEM', 'P AKAR', 'T o o l s'],
 ['PEMROSESAN', 'BAHASA', 'ALAMI'],
 ['PEMBELAJARAN', 'MESIN']]

In [317]:
all_materi_kecerdasan_buatan = []
for materi in materi_kecerdasan_buatan:
    if type(materi) == str and "PERTEMUAN" in materi:
        all_materi_kecerdasan_buatan.append(materi[15:])

    if type(materi) == str and "PERTEMUAN" not in materi:
        all_materi_kecerdasan_buatan.append(materi)

    if type(materi) == list:
        materi_ = " ".join([_materi_.replace(" ", "") for _materi_ in materi])
        materi_ = materi_.replace("Tools","")
        all_materi_kecerdasan_buatan.append(materi_)

In [318]:
all_materi_kecerdasan_buatan = [materi_.replace("  ","") for materi_ in all_materi_kecerdasan_buatan]

In [319]:
# materi_kecerdasan_buatan = ["ARTIFICIAL INTELLIGENCE", "INTELLIGENT AGENT"]
materi_kecerdasan_buatan = []
for materi_ in all_materi_kecerdasan_buatan[:5]:
    materi_kecerdasan_buatan.append(materi_)
materi_kecerdasan_buatan.append("UTS")
for materi_ in all_materi_kecerdasan_buatan[5:]:
    materi_kecerdasan_buatan.append(materi_)
materi_kecerdasan_buatan.append("UAS")

In [320]:
len(materi_kecerdasan_buatan)

14

In [321]:
tanggal_senin = output_dates[:14]
pertemuan = [i+1 for i, tanggal in enumerate(tanggal_senin)]
bulan_pertemuan = [tanggal.split()[1] for i, tanggal in enumerate(tanggal_senin)]
data_nama_dosen = ["Ratna Mufidah, M.Kom.", "Yuyun Umaidah, M.Kom", "Sofi Defiyanti, M.Kom."]
target_dir = ["DHD KECERDASAN BUATAN BU RATNA", "DHD KECERDASAN BUATAN BU YUYUN", "DHD MACHINE LEARNING BU SOFI"]
data_jabatan_akademik_dosen = ["Dosen", "Dosen", "Dosen"]
data_matkul = ["Kecerdasan Buatan", "Kecerdasan Buatan", "Machine Learning"]
data_kelas = [["4C", "4B"], ["4D"], ["6E"]]
jumlah_mahasiswa = [[27, 18], [29], [23]]
for y in range(len(jumlah_mahasiswa)):
    for x in range(len(jumlah_mahasiswa[y])):
        jumlah_mahasiswa[y][x] = str(jumlah_mahasiswa[y][x])

In [322]:
print(len(materi_kecerdasan_buatan))

14


In [323]:
len(tanggal_senin)

14

In [324]:
len(tanggal_senin)

14

In [325]:
waktu_kecerdasan_buatan_bu_ratna = n_generate_weekly_dates("19 Februari 2024", 7)

In [326]:
try:
    for dir_ in target_dir:
        clear_directory(dir_)
except:
    pass

for i, nama_dosen in enumerate(data_nama_dosen):
    if len(data_kelas[i]) == 1:
        template_file = "DHD Praktikum Tabel 1 baris.docx"
        for waktu, tanggal in enumerate(tanggal_senin):
            replacement_data_kelas = {
                "KELAS1_" : data_kelas[i][0],
                "JUMLAH_MHS_KELAS1" : jumlah_mahasiswa[i][0],
                "MHS_" : u'\u2713',
                "ASISTEN_" : "",
                "SEMESTER GENAP" : "SEMESTER GANJIL",
                "2023/2024" : "2024/2025",
            }
            replace_format = format_replacement(nama_dosen, data_jabatan_akademik_dosen[i], 
                                                data_matkul[i], pertemuan[waktu], bulan_pertemuan[waktu], 
                                                tanggal, replacement_data_kelas)
            target_output_file = '_'.join([str(y) for x,y in replace_format.items() if "2023/2024" not in x]).replace("/", "GARING").replace("__","_").replace(".","").replace(",","")
            target_output_file = f'{target_dir[i]}/{target_output_file}.docx'
            replace_placeholder(template_file, target_output_file, replace_format)
    
    if len(data_kelas[i]) == 2:
        template_file = "DHD Praktikum Tabel 2 baris.docx"
        for waktu, tanggal in enumerate(tanggal_senin):
            metode_pengajaran = "Luring"
            if waktu >= 7:
                metode_pengajaran = "Daring"

            replacement_data_kelas = {
                "MATERI_PRAKTIKUM" : materi_kecerdasan_buatan[waktu],
                "KELAS1_" : data_kelas[i][0],
                "KELAS2_" : data_kelas[i][1],
                "JUMLAH_MHS_KELAS1" : jumlah_mahasiswa[i][0],
                "JUMLAH_MHS_KELAS2" : jumlah_mahasiswa[i][1],
                "MHS_" : u'\u2713',
                "METODE_MENGAJAR" : metode_pengajaran,
                "ASISTEN_" : "",
                "SEMESTER GENAP" : "SEMESTER GANJIL",
                "2023/2024" : "2024/2025",
            }

            if waktu < 7:
                replacement_data_kelas["JAM_MASUK_1"] = "10.00"
                replacement_data_kelas["JAM_KELUAR_1"] = "12.00"
                replacement_data_kelas["JAM_MASUK_2"] = "13.00"
                replacement_data_kelas["JAM_KELUAR_2"] = "15.00"
                replacement_data_kelas["HARI_"] = "KAMIS"
                replacement_data_kelas["TANGGAL_"] = waktu_kecerdasan_buatan_bu_ratna[waktu]


            replace_format = format_replacement(nama_dosen, data_jabatan_akademik_dosen[i], 
                                                data_matkul[i], pertemuan[waktu], bulan_pertemuan[waktu], 
                                                tanggal, replacement_data_kelas)
            target_output_file = '_'.join([str(y) for x,y in replace_format.items() if "2023/2024" not in x]).replace("/", "GARING").replace("__","_").replace(".","").replace(",","")
            target_output_file = f'{target_dir[i]}/{target_output_file}.docx'
            replace_placeholder(template_file, target_output_file, replace_format)
            
    # print(target_output_file)

for i, nama_dosen in enumerate(data_nama_dosen):
    all_file = [target_dir[i]+'/'+file_ for file_ in os.listdir(target_dir[i]) if '.docx' in file_]
    all_file_dict = {int(all_file[x].split("_")[3]):y for x,y in enumerate(all_file)}
    all_file_dict = sort_dict_by_key(all_file_dict)
    all_file = [y for x, y in all_file_dict.items()]
    output_path = f"DHD_Heru Triana_{target_dir[i]}.docx"
    combine_docx(all_file, output_path)
    # print(output_path)

Semua file dan folder dalam direktori DHD KECERDASAN BUATAN BU RATNA telah dihapus.
Semua file dan folder dalam direktori DHD KECERDASAN BUATAN BU YUYUN telah dihapus.
Semua file dan folder dalam direktori DHD MACHINE LEARNING BU SOFI telah dihapus.
